In [ ]:
class KalmanFilter(object):
  # Initialize the Kalman Filter with parameters
  def __init__(self, A = None, B = None, H = None, Q = None, R = None, P = None, x0 = None):
    if(A is None or H is None):
      raise ValueError("Set proper system dynamics.")

    # number of state variables
    self.n = A.shape[1]

    # number of measurement variables
    self.m = H.shape[1]

    # assign the system matrices, using default values
    self.A = A
    self.H = H
    self.B = np.zeros((1,1)) if B is None else B

    # covariance is often a diagonal matrix since variables are independent
    self.Q = np.eye(self.n) if Q is None else Q # process noise covariance matrix
    self.R = np.eye(self.n) if R is None else R # measurement noise covariance matrix
    self.P = np.eye(self.n) if P is None else P # the initial error estimate covariance matrix
    self.x = np.zeros((self.n, 1)) if x0 is None else x0 # the initial state estimate vector

  # Prediction step of the Kalman filter
  def predict(self, u = None):
    if u is None: u = np.zeros((self.B.shape[1],1))

    self.x = (self.A @ self.x) + (self.B @ u) # predict the next state estimate
    self.P = ((self.A @ self.P) @ self.A.T) + self.Q # predict the next estimate error covariance
    return self.x

  # Update step of the Kalman Filter
  def update(self, z):
    S = self.R + (self.H @ (self.P @ self.H.T)) # calculate the measurement residual covariance
    K = sp.linalg.cho_solve(sp.linalg.cho_factor(S.T),(self.H @ self.P.T)).T # Kalman gain using a Cholesky solver
    y = z - (self.H @ self.x) # calculate the measurement residual
    self.x = self.x + (K @ y) # update the state estimate

    # update the estimate error covariance
    I = np.eye(self.n)
    self.P = (((I - (K @ self.H)) @ self.P) @ ((I - (K @ self.H)).T))+(K @ (self.R @ K.T))
    return K